## Visualization Mapping

In [0]:
import json
import yaml
import re
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
from datetime import datetime
import os
import uuid

# Configuration
CATALOG = "dbx_migration_poc"
SCHEMA = "dbx_migration_ts"
TML_VOLUME = "lv_dashfiles_ak"
LVDASH_VOLUME = "lvdash_files_ak_out"

dbutils.widgets.text("tml_file", "")
tml_file = dbutils.widgets.get("tml_file")

TML_INPUT_PATH = f"/Volumes/dbx_migration_poc/dbx_migration_ts/lv_dashfiles_ak/liveboard/{tml_file}"
LVDASH_OUTPUT_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{LVDASH_VOLUME}/"

TRACKER_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_tracker"
SUMMARY_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_summary"
FAILURE_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_failures"
MAPPING_TABLE = f"{CATALOG}.{SCHEMA}.tml_dbx_metadata_mapping"

LIVEBOARD_FILE = f"{TML_INPUT_PATH}"

raw_name = os.path.basename(LIVEBOARD_FILE).split('.')[0]
asset_name = re.sub(r'[\s\-]+', '_', raw_name)

# Configuration tables
CHART_TYPE_MAPPING_TABLE = f"{CATALOG}.{SCHEMA}.chart_type_mappings"
WIDGET_SIZE_CONFIG_TABLE = f"{CATALOG}.{SCHEMA}.widget_size_config"
EXPRESSION_TRANSFORM_TABLE = f"{CATALOG}.{SCHEMA}.expression_transformations"
SCALE_TYPE_DETECTION_TABLE = f"{CATALOG}.{SCHEMA}.scale_type_detection"
COLUMN_DETAILS_TABLE = f"{CATALOG}.{SCHEMA}.{asset_name}_support_viz_column_details"
FILTER_MAPPING_TABLE = f"{CATALOG}.{SCHEMA}.{asset_name}_filter_details"
VIZ_FILTER_TABLE = f"{CATALOG}.{SCHEMA}.{asset_name}_support_viz_filter_metadata"

def validate_configuration():
    print("--- Validating configuration ---")
    error_found = False
    for var_name, var_value in [("CATALOG", CATALOG), ("SCHEMA", SCHEMA), ("TML_VOLUME", TML_VOLUME), ("LVDASH_VOLUME", LVDASH_VOLUME)]:
        if "/" in var_value or "\\" in var_value:
            print(f"ERROR: '{var_name}' contains a slash. Must be a single name.")
            error_found = True
    
    if not TML_INPUT_PATH.startswith("/Volumes/"):
        print(f"WARNING: 'TML_INPUT_PATH' does not look like a Volume path.")
        error_found = True
    
    if error_found:
        raise ValueError("Invalid configuration. Please review errors above.")
    else:
        print("Configuration looks good.")

validate_configuration()

def load_viz_filter_metadata():
    try:
        if not spark.catalog.tableExists(VIZ_FILTER_TABLE): return pd.DataFrame()
        df = spark.table(VIZ_FILTER_TABLE).toPandas()
        print(f"✓ Loaded {len(df)} viz filter records from {VIZ_FILTER_TABLE}")
        return df
    except: return pd.DataFrame()

VIZ_FILTER_DATA = load_viz_filter_metadata()

def load_chart_type_mappings():
    """Load chart type mappings from configuration table - REQUIRED"""
    try:
        df = spark.table(CHART_TYPE_MAPPING_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {CHART_TYPE_MAPPING_TABLE} is empty. Please run the configuration setup notebook first.")
        
        mapping = {}
        for _, row in df.iterrows():
            tml_type = row['tml_chart_type']
            widget_type = row['widget_type']
            mapping[tml_type] = widget_type if pd.notna(widget_type) else None
        
        print(f"✓ Loaded {len(mapping)} chart type mappings from {CHART_TYPE_MAPPING_TABLE}")
        return mapping
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {CHART_TYPE_MAPPING_TABLE}

Details: {str(e)}

SOLUTION: You must run the configuration setup notebook to create the required tables:
1. Open the 'Configuration Tables Setup' notebook
2. Run all cells to create: chart_type_mappings, widget_size_config, expression_transformations, scale_type_detection
3. Then re-run this converter

Configuration tables are REQUIRED and no fallback mappings are available.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {CHART_TYPE_MAPPING_TABLE} not found or cannot be loaded. Run configuration setup first.") from e

def load_expression_transformations():
    """Load expression transformation rules from configuration table - REQUIRED"""
    try:
        df = spark.table(EXPRESSION_TRANSFORM_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {EXPRESSION_TRANSFORM_TABLE} is empty. Please run the configuration setup notebook first.")
        
        transformations = []
        for _, row in df.iterrows():
            transformations.append({
                'pattern': row['tml_pattern'],
                'target': row['target_expression']
            })
        print(f"✓ Loaded {len(transformations)} expression transformations from {EXPRESSION_TRANSFORM_TABLE}")
        return transformations
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {EXPRESSION_TRANSFORM_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {EXPRESSION_TRANSFORM_TABLE} not found. Run configuration setup first.") from e

def load_column_details():
    try:
        df = spark.table(COLUMN_DETAILS_TABLE).toPandas()
        if len(df) == 0:
            print(f"WARNING: Column details table {COLUMN_DETAILS_TABLE} is empty.")
            return pd.DataFrame()
        print(f"✓ Loaded {len(df)} column detail records from {COLUMN_DETAILS_TABLE}")
        return df
    except Exception as e:
        print(f"WARNING: Could not load column details table: {e}")
        return pd.DataFrame()

def load_scale_type_rules():
    """Load scale type detection rules from configuration table - REQUIRED"""
    try:
        df = spark.table(SCALE_TYPE_DETECTION_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {SCALE_TYPE_DETECTION_TABLE} is empty. Please run the configuration setup notebook first.")
        
        rules = []
        for _, row in df.iterrows():
            rules.append({
                'pattern': row['field_pattern'],
                'scale_type': row['scale_type']
            })
        print(f"✓ Loaded {len(rules)} scale type rules from {SCALE_TYPE_DETECTION_TABLE}")
        return rules
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {SCALE_TYPE_DETECTION_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {SCALE_TYPE_DETECTION_TABLE} not found. Run configuration setup first.") from e

def load_widget_size_config():
    """Load widget size configuration from database table - REQUIRED"""
    try:
        df = spark.table(WIDGET_SIZE_CONFIG_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {WIDGET_SIZE_CONFIG_TABLE} is empty. Please run the configuration setup notebook first.")
        
        size_map = {}
        for _, row in df.iterrows():
            size_map[row['size_category']] = {
                'width': row['width'],
                'height': row['height']
            }
        print(f"✓ Loaded {len(size_map)} widget size configurations from {WIDGET_SIZE_CONFIG_TABLE}")
        return size_map
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {WIDGET_SIZE_CONFIG_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {WIDGET_SIZE_CONFIG_TABLE} not found. Run configuration setup first.") from e

def load_filter_mappings():
    """Load filter definitions from configuration table - REQUIRED"""
    try:
        # Check if table exists first to avoid hard crash on spark.table()
        if not spark.catalog.tableExists(FILTER_MAPPING_TABLE):
             print(f"WARNING: Filter table {FILTER_MAPPING_TABLE} not found. Filters will be skipped.")
             return pd.DataFrame()
        df = spark.table(FILTER_MAPPING_TABLE).toPandas()
        
        if len(df) == 0:
            print(f"WARNING: Configuration table {FILTER_MAPPING_TABLE} is empty. No filters will be generated.")
            return pd.DataFrame()
        
        # We return the DataFrame directly because the filter logic needs to query it
        # dynamically based on the filename later in the script.
        print(f"✓ Loaded {len(df)} filter mappings from {FILTER_MAPPING_TABLE}")
        return df
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load configuration table: {FILTER_MAPPING_TABLE}
Details: {str(e)}
SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        # If filters are strictly required, uncomment the next line.
        # Otherwise, returning an empty DF allows the script to continue without filters.
        # raise RuntimeError(f"Required configuration table {FILTER_MAPPING_TABLE} not found.") from e
        return pd.DataFrame()

# Load all configuration data
TML_TO_LVDASH_MAPPING = load_chart_type_mappings()
WIDGET_SIZE_MAP = load_widget_size_config()
EXPRESSION_TRANSFORMATIONS = load_expression_transformations()
SCALE_TYPE_RULES = load_scale_type_rules()
COLUMN_DETAILS_DATA = load_column_details()
FILTER_DATA = load_filter_mappings()


def load_mapping_data():
    try:
        mapping_df = spark.table(MAPPING_TABLE).toPandas()
        print(f"Loaded {len(mapping_df)} mappings from {MAPPING_TABLE}")
        
        unmapped = mapping_df[
            (mapping_df['databricks_table_name_ToBeFilled'].isna()) | 
            (mapping_df['databricks_table_name_ToBeFilled'] == '')
        ]
        
        if len(unmapped) > 0:
            print(f"\nWARNING: {len(unmapped)} visualizations are not mapped to Databricks tables:")
            for _, row in unmapped.head(10).iterrows():
                print(f"  - {row['tml_file']}: {row['visualization_name']}")
            if len(unmapped) > 10:
                print(f"  ... and {len(unmapped) - 10} more")
            print("\nThese visualizations will use TML table names directly.")
        
        return mapping_df
    except Exception as e:
        print(f"WARNING: Could not load mapping table: {e}")
        print("Conversion will proceed using TML table names directly.")
        return None

MAPPING_DATA = load_mapping_data()

def setup_environment():
    def ensure_schema_exists(catalog, schema):
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`")
    
    def ensure_volume_exists(catalog, schema, volume):
        try:
            spark.sql(f"DESCRIBE VOLUME `{catalog}`.`{schema}`.`{volume}`")
        except Exception:
            print(f"  Creating volume {volume}...")
            spark.sql(f"CREATE VOLUME IF NOT EXISTS `{catalog}`.`{schema}`.`{volume}`")
    
    def ensure_table_exists(full_table_name, table_type):
        parts = full_table_name.split('.')
        catalog = parts[0]
        schema = parts[1]
        table_name = parts[2]
        
        try:
            spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{table_name}`")
            print(f"  Dropped existing table: {table_name}")
        except Exception:
            pass
        
        if table_type == "tracker":
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING, 
                    widget_name STRING, 
                    tml_type STRING, 
                    lvdash_type STRING, 
                    status STRING, 
                    available_fields STRING, 
                    missing_fields STRING, 
                    unmapped_properties STRING, 
                    notes STRING, 
                    conversion_timestamp TIMESTAMP
                ) USING DELTA
            """
        elif table_type == "failure":
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING,
                    error_type STRING,
                    error_message STRING,
                    stack_trace STRING,
                    failure_timestamp TIMESTAMP
                ) USING DELTA
            """
        else:
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING, 
                    lvdash_file STRING, 
                    status STRING, 
                    num_datasets INT, 
                    num_pages INT, 
                    num_widgets INT, 
                    conversion_timestamp TIMESTAMP
                ) USING DELTA
            """
        
        spark.sql(create_sql)
        print(f"  Created table: {table_name}")
    
    print("\n--- Setting up environment ---")
    ensure_schema_exists(CATALOG, SCHEMA)
    ensure_volume_exists(CATALOG, SCHEMA, TML_VOLUME)
    ensure_volume_exists(CATALOG, SCHEMA, LVDASH_VOLUME)
    ensure_table_exists(TRACKER_TABLE, "tracker")
    ensure_table_exists(SUMMARY_TABLE, "summary")
    ensure_table_exists(FAILURE_TABLE, "failure")
    print("--- Environment ready ---\n")

def generate_unique_id(length=8):
    return uuid.uuid4().hex[:length]

def parse_tml_file(file_path):
    try:
        content = dbutils.fs.head(file_path, 10 * 1024 * 1024)
        try:
            return yaml.safe_load(content)
        except yaml.YAMLError:
            return json.loads(content)
    except Exception as e:
        raise ValueError(f"Failed to parse TML file: {str(e)}")

def extract_colors_from_tml(chart_data):
    try:
        client_state = chart_data.get('client_state_v2', '{}')
        if isinstance(client_state, str):
            client_state = json.loads(client_state)
        
        system_colors = client_state.get('systemSeriesColors', [])
        if system_colors:
            colors = []
            for item in system_colors:
                if item.get('color') and item['color'] not in colors:
                    colors.append(item['color'])
            if colors:
                return colors
    except Exception:
        pass
    
    return ["#2E75F0", "#06BF7F", "#FCC838", "#48D1E0", "#71A1F4", "#8C62F5"]

def infer_scale_type(column_name):
    """Infer scale type using configuration rules"""
    name = column_name.lower()
    
    # Use configuration rules if available
    if SCALE_TYPE_RULES:
        for rule in SCALE_TYPE_RULES:
            pattern = rule['pattern'].lower()
            # Remove wildcards and check if pattern is in the column name
            pattern_parts = pattern.replace('*_', '').replace('_*', '').split(',')
            for part in pattern_parts:
                part = part.strip()
                if part in name:
                    return rule['scale_type']
    
    # Fallback to default logic
    if any(t in name for t in ['date', 'time', 'year', 'month', 'day', 'timestamp', 'week']):
        return 'temporal'
    
    if any(t in name for t in ['total', 'sum', 'count', 'avg', 'average', 'min', 'max', 'revenue', 'price', 'quantity', 'amount', 'sales', 'number']):
        return 'quantitative'
    
    return 'categorical'

def clean_field_name(field_name):
    if not field_name:
        return ""
    cleaned = re.sub(
        r'^(Total |sum\(|count\(|avg\(|min\(|max\(|Unique Number of )',
        '',
        field_name,
        flags=re.IGNORECASE
    )
    cleaned = re.sub(r'\)$', '', cleaned)
    return cleaned.strip()

def sanitize_alias_name(alias_name):
  if not alias_name:
    return ""
  safe_name = re.sub(r'\W+', '_', alias_name)
  sanitized_alias = safe_name.strip('_')
  return sanitized_alias if sanitized_alias else "_"

class ConversionTracker:
    def __init__(self):
        self.records = []
    
    def add_record(self, tml_file, widget_name, tml_type, lvdash_type, status, available, missing, unmapped_props, notes=""):
        self.records.append({
            'tml_file': tml_file, 
            'widget_name': widget_name, 
            'tml_type': tml_type,
            'lvdash_type': lvdash_type, 
            'status': status,
            'available_fields': ', '.join(available), 
            'missing_fields': ', '.join(missing),
            'unmapped_properties': ', '.join(unmapped_props),
            'notes': notes, 
            'conversion_timestamp': datetime.now()
        })

class TMLToLVDASHConverter:
    # Update __init__ arguments
    def __init__(self, tml_data, tml_filename, tracker, mapping_data=None, column_details_data=None, viz_filter_data=None):
        self.tml_data = tml_data
        self.tml_filename = tml_filename
        self.tracker = tracker
        self.mapping_data = mapping_data
        self.column_details_data = column_details_data if column_details_data is not None else pd.DataFrame()
        # NEW: Store viz filter data
        self.viz_filter_data = viz_filter_data if viz_filter_data is not None else pd.DataFrame()
        
    def _get_mapping_for_viz(self, viz_id):
        """Get mapping for a specific visualization ID, returns Series or None"""
        if self.mapping_data is None or len(self.mapping_data) == 0:
            return None
        
        try:
            mapping = self.mapping_data[
                (self.mapping_data['tml_file'] == self.tml_filename) &
                (self.mapping_data['visualization_id'] == viz_id)
            ]
            
            # Always return a Series (single row) or None
            if not mapping.empty:
                return mapping.iloc[0]  
            return None
        except Exception as e:
            print(f"  WARNING: Error getting mapping for viz {viz_id}: {e}")
            return None
        
    def _get_column_details_for_viz(self, viz_id):
        if self.column_details_data is None or self.column_details_data.empty:
            return {}
        try:
            # 1. Filter column details for this Viz
            viz_columns = self.column_details_data[self.column_details_data['VizID'] == viz_id]
            if viz_columns.empty:
                return {}
            
            # 2. Filter metadata for this Viz (NEW)
            viz_filters = pd.DataFrame()
            if hasattr(self, 'viz_filter_data') and not self.viz_filter_data.empty:
                 viz_filters = self.viz_filter_data[self.viz_filter_data['viz_id'] == viz_id]

            column_dict = {}
            for _, row in viz_columns.iterrows():
                column_name = row['ColumnName']
                sanitized_name = row['Santized_Column']
                
                # 3. Lookup Filter Details (NEW)
                filter_val = None
                if not viz_filters.empty:
                    match = viz_filters[viz_filters['Sanitized_Column'] == sanitized_name]
                    if not match.empty:
                        filter_val = match.iloc[0]['Filter_Details']

                column_dict[column_name] = {
                    'model_base_column': row['ModelBaseColumn'],
                    'aggregation': row['Aggregation'] if pd.notna(row['Aggregation']) else None,
                    'expression': row['Expression'] if pd.notna(row['Expression']) else None,
                    'sanitized': sanitized_name,
                    'filter_condition': filter_val # Store the filter like .false or .weekly
                }
            return column_dict
        except Exception as e:
            print(f"  WARNING: Error getting column details for viz {viz_id}: {e}")
            return {}
    def _detect_filter_widget_type(self, column_name, is_single_value=False):
        """
        Determines the correct Databricks widget type based on column name and single-value flag.
        """
        if not column_name: return "filter-multi-select"
        col_lower = column_name.lower()
        
        # 1. Check for Date/Time
        is_date = any(x in col_lower for x in ['date', 'time', 'created', 'closed', 'timestamp'])
        
        if is_date:
            if is_single_value:
                return "filter-date-picker"       # Single Date
            else:
                return "filter-date-range-picker" # Date Range (Default)
        
        # 2. Check for Non-Date Single/Multi
        if is_single_value:
            return "filter-single-select"
            
        return "filter-multi-select"
    def _create_filter_widgets(self, datasets_map):
        """
        Creates filter widgets with SQL Scanning, Deduplication, and Default Values.
        """
        if FILTER_DATA is None or FILTER_DATA.empty: return []
        
        # 1. Fuzzy match filename
        clean_filename = self.tml_filename.replace('.tml', '').replace('.json', '').strip()
        if 'tml_file' in FILTER_DATA.columns:
             file_filters = FILTER_DATA[FILTER_DATA['tml_file'].astype(str).str.contains(clean_filename, case=False, regex=False)]
        else:
             file_filters = FILTER_DATA
            
        if file_filters.empty: return []
        if not datasets_map: return []
        
        # 2. Get the Unified Dataset & SQL
        target_dataset_obj = next(iter(datasets_map.values()))
        target_dataset_id = target_dataset_obj['name']
        dataset_sql = " ".join(target_dataset_obj.get('queryLines', [])).lower()
        
        filter_widgets = []
        seen_aliases = set() # Deduplication check
        
        print(f"\n--- Creating Filters (Linked to: {target_dataset_id}) ---")
        
        # --- Helper: Find Alias in SQL ---
        def find_alias_in_sql(col_name):
            import re
            clean_col = col_name.lower().replace('`', '').strip()
            if "::" in clean_col: clean_col = clean_col.split("::")[-1].strip()
            
            # 1. Exact Match: "Column Name" AS Alias
            pattern = r"['`\"]?" + re.escape(clean_col) + r"['`\"]?\s+as\s+['`\"]?([\w_]+)['`\"]?"
            match = re.search(pattern, dataset_sql)
            if match: return match.group(1)
            
            # 2. Base Match: "Name" AS Alias (ignores prefixes)
            short_col = clean_col.split('tbl_')[-1] if 'tbl_' in clean_col else clean_col
            pattern_short = r"['`\"]?" + re.escape(short_col) + r"['`\"]?\s+as\s+['`\"]?([\w_]+)['`\"]?"
            match_short = re.search(pattern_short, dataset_sql)
            if match_short: return match_short.group(1)
            
            return sanitize_alias_name(clean_col)
        # ---------------------------------

        for index, row in file_filters.iterrows():
            try:
                display_name = row.get('display_name', 'Filter')
                
                # Get Raw Column
                if 'Filter_Column' in row and pd.notna(row['Filter_Column']):
                    raw_col_name = row['Filter_Column']
                else:
                    phys_id = row.get('Physical_Column_ID', '')
                    raw_col_name = phys_id.split('::')[-1] if '::' in str(phys_id) else phys_id
                
                raw_col_name = raw_col_name.replace('[', '').replace(']', '').strip()
                
                # [CRITICAL FIX] Find Actual SQL Alias
                sql_alias = find_alias_in_sql(raw_col_name)
                
                # ... (Existing code matches lines 606-607)
                if sql_alias in seen_aliases: continue
                seen_aliases.add(sql_alias)

                # --- NEW LOGIC START: Process Default Values ---
                default_selection_spec = {}
                raw_values = str(row.get('Values', '')).strip()

                # Check if values exist and are not null/nan
                if raw_values and raw_values.lower() not in ['nan', 'none', '', 'null']:
                    # Regex to extract values inside single quotes: ('Value') -> Value
                    # This handles the user requirement: "('4C Portal...')" -> "4C Portal..."
                    extracted_values = re.findall(r"'([^']*)'", raw_values)

                    # Fallback: If no quotes found but text exists (e.g. numeric (1, 2)), strip parens and split
                    if not extracted_values:
                        cleaned_raw = raw_values.replace('(', '').replace(')', '')
                        extracted_values = [x.strip() for x in cleaned_raw.split(',') if x.strip()]

                    if extracted_values:
                        default_selection_spec = {
                            "defaultSelection": {
                                "values": {
                                    "dataType": "STRING", # Defaulting to STRING for categorical filters
                                    "values": [{"value": v} for v in extracted_values]
                                }
                            }
                        }
                # --- NEW LOGIC END ---

                # Determine Type
                is_single = str(row.get('is_single_value', 'false')).lower() in ['true', 'yes', '1']
                widget_type = self._detect_filter_widget_type(raw_col_name, is_single)
                
                widget_id = f"filter_{generate_unique_id()}"
                query_id = f"query_{generate_unique_id()}"
                
                # Build JSON
                filter_spec = {
                    "version": 2,
                    "widgetType": widget_type,
                    "encodings": {
                        "fields": [
                            {"fieldName": sql_alias, "queryName": query_id}
                        ]
                    },
                    "frame": {
                        "showTitle": True,
                        "title": display_name
                    }
                }

                # Apply default selection if found
                if default_selection_spec:
                    filter_spec["selection"] = default_selection_spec

                filter_widget = {
                    "viz_id": f"filter_{index}",
                    "is_filter": True,
                    "widget": {
                        "name": widget_id,
                        "queries": [
                            {
                                "name": query_id,
                                "query": {
                                    "datasetName": target_dataset_id,
                                    "fields": [
                                        {"name": sql_alias, "expression": f"`{sql_alias}`"},
                                        {"name": f"{sql_alias}_associativity", "expression": "COUNT_IF(`associative_filter_predicate_group`)"}
                                    ],
                                    "disaggregated": False
                                }
                            }
                        ],
                        "spec": filter_spec
                    }
                }
                
                filter_widgets.append(filter_widget)
                # ... (Rest of the function remains the same)
                print(f"  ✓ Created {widget_type}: {display_name} -> Alias: {sql_alias}")
                
            except Exception as e:
                print(f"  ERROR creating filter row {index}: {e}")
                
        return filter_widgets
    # ----------------------------------------

    def convert(self):
        liveboard = self.tml_data.get('liveboard')
        if not liveboard:
            raise ValueError("TML file missing 'liveboard' root key.")

        visualizations = liveboard.get('visualizations', [])
        if not visualizations:
            raise ValueError("No visualizations found in liveboard.")
        
        # Track unique datasets by their name (supports reuse)
        datasets_map = {}  
        dataset_usage = {}  
        widgets = []

        for viz in visualizations:
            try:
                viz_id = viz.get('id')
                viz_name = viz.get('answer', {}).get('name', 'Unknown')
                
                # Check if this viz should use a common/shared dataset
                mapping = self._get_mapping_for_viz(viz_id)
                column_details = self._get_column_details_for_viz(viz_id)
                print("Check1", mapping)
                
                if mapping is not None:
                    common_ds = mapping.get('common_dataset_name')
                    if pd.notna(common_ds) and common_ds and str(common_ds).strip() != '' and str(common_ds).lower() != 'null':
                        # USE SHARED DATASET from mapping table
                        dataset_name = common_ds
                        if dataset_name not in datasets_map:
                            dataset = self._create_shared_dataset(viz, mapping)
                            datasets_map[dataset_name] = dataset
                            dataset_usage[dataset_name] = []
                            print(f"  INFO: Created shared dataset '{dataset_name}' for visualization '{viz_name}'")
                        else:
                            print(f"  INFO: Reusing shared dataset '{dataset_name}' for visualization '{viz_name}'")
                        
                        dataset_usage[dataset_name].append(viz_name)
                    else:
                        # CREATE UNIQUE DATASET per visualization
                        dataset = self._create_dataset(viz)
                        dataset_name = dataset['name']
                        datasets_map[dataset_name] = dataset
                        dataset_usage[dataset_name] = [viz_name]
                        print(f"  INFO: Created unique dataset '{dataset_name}' for visualization '{viz_name}'")
                else:
                    # CREATE UNIQUE DATASET per visualization (no mapping found)
                    dataset = self._create_dataset(viz)
                    dataset_name = dataset['name']
                    datasets_map[dataset_name] = dataset
                    dataset_usage[dataset_name] = [viz_name]
                    print(f"  INFO: Created unique dataset '{dataset_name}' for visualization '{viz_name}'")
                        
                # Create widget using the appropriate dataset name
                widget_data = self._create_widget(viz, dataset_name)
                if isinstance(widget_data, list):
                    for w in widget_data:
                        if isinstance(w, dict):
                            widgets.append(w)
                        else:
                            print(f"  WARNING: Skipping invalid widget object of type {type(w)}")
                elif isinstance(widget_data, dict):
                    widgets.append(widget_data)
                else:
                    print(f"  WARNING: _create_widget returned unexpected type {type(widget_data)} for viz {viz_id}")
                
            except Exception as viz_error:
                viz_name = viz.get('answer', {}).get('name', 'Unknown')
                print(f"  WARNING: Failed to convert visualization '{viz_name}': {viz_error}")
                import traceback
                traceback.print_exc()
                self.tracker.add_record(
                    self.tml_filename,
                    viz_name,
                    viz.get('answer', {}).get('chart', {}).get('type', 'UNKNOWN'),
                    'ERROR',
                    'ERROR',
                    [],
                    [],
                    [],
                    f"Conversion failed: {str(viz_error)[:500]}"
                )
                continue
        try:
            filter_widgets = self._create_filter_widgets(datasets_map)
            if filter_widgets:
                widgets.extend(filter_widgets)
                print(f"  INFO: Added {len(filter_widgets)} filter widgets.")
        except Exception as e:
            print(f"  ERROR generating filters: {e}")

        if not widgets:
            raise ValueError("No widgets could be converted successfully.")

        # Log dataset reuse statistics
        print(f"\n--- Dataset Reuse Summary ---")
        for ds_name, viz_list in dataset_usage.items():
            if len(viz_list) > 1:
                print(f"  ✓ Shared dataset '{ds_name}' used by {len(viz_list)} visualizations: {', '.join(viz_list[:3])}{'...' if len(viz_list) > 3 else ''}")
        print(f"Total datasets created: {len(datasets_map)}")
        print(f"Total widgets created: {len(widgets)}")
        
        # Check widget structure
        print(f"\n--- Widget Structure Check ---")
        for i, w in enumerate(widgets):
            if not isinstance(w, dict):
                print(f"  ERROR: Widget at index {i} is type {type(w)}, not dict!")
            elif 'widget' not in w:
                print(f"  WARNING: Widget at index {i} missing 'widget' key")
            else:
                print(f"  ✓ Widget {i}: viz_id={w.get('viz_id')}, type={w.get('widget', {}).get('spec', {}).get('widgetType')}")
        print("="*50 + "\n")

        return {
            "datasets": list(datasets_map.values()),
            "pages": [{
                "name": generate_unique_id(),
                "displayName": liveboard.get('name', 'Converted Dashboard'),
                "layout": self._create_layout(liveboard.get('layout'), widgets),
                "pageType": "PAGE_TYPE_CANVAS"
            }]
        }

    def _get_sql_filter(self, col_expression, filter_str):
        """
        Converts TML shorthand into SQL conditions.
        Applies lower() to string comparisons to ensure case-insensitive matching
        (e.g. matching 'work in progress' to 'Work in Progress').
        """
        if not filter_str or pd.isna(filter_str) or str(filter_str).lower() in ['nan', 'null', 'none']:
            return None
        s = str(filter_str).strip()

        # 1. Handle Null Checks
        if "!= '{null}'" in s: return f"{col_expression} IS NOT NULL"
        if "= '{null}'" in s: return f"{col_expression} IS NULL"

        # 2. Handle Lists (comma separated) -> IN clause
        # Example: .on-hold, .Work in Progress -> lower(Col) IN ('on-hold', 'work in progress')
        if "," in s:
            parts = []
            has_strings = False
            for p in s.split(','):
                val = p.strip()
                if val.startswith('.'): val = val[1:] # Remove dot
                
                # Check types
                if val.lower() in ['true', 'false']:
                    parts.append(val.upper()) # TRUE/FALSE (No quotes)
                elif re.match(r'^\d+(\.\d+)?$', val):
                    parts.append(val) # Number (No quotes)
                else:
                    # String: Mark as string list and lowercase the value
                    has_strings = True
                    parts.append(f"'{val.lower()}'")
            
            if has_strings:
                # Apply lower() to the column for case-insensitive match
                return f"lower({col_expression}) IN ({', '.join(parts)})"
            else:
                return f"{col_expression} IN ({', '.join(parts)})"

        # 3. Handle Single Dot Values
        if s.startswith('.'):
            val = s[1:]
            # Booleans
            if val.lower() == 'true': return f"{col_expression} = true"
            if val.lower() == 'false': return f"{col_expression} = false"
            # Numbers
            if re.match(r'^\d+(\.\d+)?$', val): return f"{col_expression} = {val}"
            
            # Strings -> Case Insensitive Equality
            return f"lower({col_expression}) = '{val.lower()}'"

        # 4. Handle Standard Operators (>, =, !=)
        # Regex to separate Operator from Value
        match = re.match(r'^(=|!=|<>|>|<|>=|<=)\s*(.*)', s)
        if match:
            op = match.group(1)
            val = match.group(2).strip()
            
            # If value is a string (quoted or text), handle case insensitivity
            if val.startswith("'") or val.startswith('"'):
                clean_val = val.strip("'\"")
                return f"lower({col_expression}) {op} '{clean_val.lower()}'"
            elif not re.match(r'^\d+(\.\d+)?$', val) and val.lower() not in ['true', 'false', 'null']:
                 # Unquoted string value found
                 return f"lower({col_expression}) {op} '{val.lower()}'"
                 
            # Boolean/Numeric operators return as is
            return f"{col_expression} {s}"
            
        return None

    def _translate_search_query_to_sql(self, viz):
        answer = viz.get('answer', {})
        query = answer.get('search_query', '')
        viz_id = viz.get('id')
        print("Viz_id", viz_id)
        print("Query", query)
        print("Answer", answer)

        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id)
        
        # --- START: TABLE NAME LOGIC ---
      
        if mapping is not None and pd.notna(mapping.get('common_sql_query')) and mapping.get('common_sql_query'):
            # STRATEGY 1: Use the pre-built "common_sql_query".
            print(f"  INFO: Using 'common_sql_query' from mapping for viz {viz_id}")
            return mapping['common_sql_query'].strip().split('\n')

        if mapping is not None and pd.notna(mapping.get('search_query_final')) and mapping.get('search_query_final'):
            # STRATEGY 2: Use the pre-built "search_query_final".
            print(f"  INFO: Using 'search_query_final' from mapping for viz {viz_id}")
            return mapping['search_query_final'].strip().split('\n')
        
        # --- STRATEGY 3: Auto-generate SQL (The Fallback) ---
        print(f"  WARNING: No common/final SQL found. Auto-generating SQL for viz {viz_id}.")
        
        if mapping is not None and mapping.get('databricks_table_name_ToBeFilled'):
            table_name = mapping['databricks_table_name_ToBeFilled']
            print(f"  INFO: Using mapped table name: {table_name}")
        else:
            table_name = answer.get('tables', [{}])[0].get('name', 'your_source_table')
            print(f"  INFO: Using TML table name (fallback): {table_name}")
        


        column_mapping = {}
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                column_mapping = {}
        
        answer_cols = [col.get('name') for col in answer.get('answer_columns', []) if col.get('name')]
        
        select_clauses = []
        group_by_cols = []
        where_clauses = []
        
        for i, col_name in enumerate(answer_cols, 1):
            if not col_name:
                continue
            
            base_field = clean_field_name(col_name)
            mapped_field = column_mapping.get(base_field, base_field)
            mapped_col_name = column_mapping.get(col_name, col_name)
            sanitized_alias = sanitize_alias_name(mapped_col_name)
            sanitized_base_alias = sanitize_alias_name(base_field)
            
            # --- NEW: Apply Filters from Metadata Table ---
            if not self.viz_filter_data.empty:
                filter_row = self.viz_filter_data[
                    (self.viz_filter_data['viz_id'] == viz_id) & 
                    (self.viz_filter_data['Sanitized_Column'] == sanitized_alias)
                ]
                
                if not filter_row.empty:
                    filter_val = filter_row.iloc[0]['Filter_Details']
                    col_expr = f"`{sanitized_alias}`"
                    sql_cond = self._get_sql_filter(col_expr, filter_val)
                    if sql_cond:
                        where_clauses.append(sql_cond)
                        print(f"  INFO: Added filter for {sanitized_alias}: {sql_cond}")
            # ---------------------------------------------

            expression_found = False
            for transform in EXPRESSION_TRANSFORMATIONS:
                pattern_str = transform['pattern'] 
                target_sql = transform['target'] 
                regex_pattern = re.escape(pattern_str).replace(r'\{field\}', r'(.+)')
                
                match = re.match(regex_pattern, col_name, re.IGNORECASE)
                
                if match:
                    field_inside_func = match.group(1)
                    mapped_field_inside = column_mapping.get(field_inside_func, field_inside_func)
                    sql_expression = target_sql.format(field=mapped_field_inside)
                    sql_alias = sanitize_alias_name(column_mapping.get(col_name, col_name))                    
                    select_clauses.append(f"  {sql_expression} AS {sql_alias}")
                    group_by_cols.append(str(i))
                    expression_found = True
                    print(f"  INFO: Transformed '{col_name}' -> '{sql_expression} AS {sql_alias}'")
                    break 
            
            if not expression_found:
                # No transformation matched. This is the fallback logic.
                if 'Unique Number of' in col_name or 'unique number of' in col_name.lower():
                     match_inner = re.match(r"Unique Number of \((.+)\)", col_name, re.IGNORECASE)
                     if match_inner:
                         field_inside = match_inner.group(1)
                         mapped_field_inside = column_mapping.get(field_inside, field_inside)
                         select_clauses.append(f"  COUNT(DISTINCT {mapped_field_inside}) AS {sanitized_alias}")
                     else:
                         select_clauses.append(f"  COUNT(DISTINCT {mapped_field}) AS {sanitized_alias}")
                
                elif any(p in col_name for p in ['Total ', 'sum(', 'count(', 'Sum(', 'Count(']):
                    agg = 'COUNT' if 'count' in col_name.lower() else 'SUM'
                    select_clauses.append(f"  {agg}({mapped_field}) AS {sanitized_base_alias}")
                
                else:
                    select_clauses.append(f"  {mapped_field} AS {sanitized_base_alias}")
                    group_by_cols.append(str(i))

        if not select_clauses:
            return [f"SELECT '' AS placeholder FROM {table_name}"]

        if ". 'this month'" in query:
            where_clauses.append("DATE >= DATE_TRUNC('MONTH', CURRENT_TIMESTAMP())")
        if ". 'last 30 days'" in query:
            where_clauses.append("DATE >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)")

        order_by = ""
        limit = ""
        if 'top 10' in query.lower():
            measure_idx = next((i for i, col in enumerate(answer_cols, 1) if any(p in col for p in ['Total', 'Unique', 'sum(', 'count('])), 1)
            order_by = f" ORDER BY {measure_idx} DESC"
            limit = " LIMIT 10"

        sql_parts = ["SELECT", ",\n".join(select_clauses), f" FROM {table_name} "]
        
        if where_clauses:
            sql_parts.append(f" WHERE {' AND '.join(where_clauses)} ")
        
        has_agg = any(agg in ' '.join(select_clauses).upper() for agg in ['SUM(', 'COUNT(', 'AVG('])
        if group_by_cols and has_agg:
            sql_parts.append(f" GROUP BY {', '.join(group_by_cols)} ")
        
        if order_by:
            sql_parts.append(order_by)
        if limit:
            sql_parts.append(limit)
        
        return '\n'.join(sql_parts).split('\n')

    def _create_dataset(self, viz):
        answer = viz.get('answer', {})
        print("create dataset inside", f"ds_{viz.get('id', generate_unique_id())}" , " ", answer.get('name', 'Untitled Dataset')," ", self._translate_search_query_to_sql(viz)  )


        return {
            "name": f"ds_{viz.get('id', generate_unique_id())}",
            "displayName": answer.get('name', 'Untitled Dataset'),
            "queryLines": self._translate_search_query_to_sql(viz)
        }

    def _extract_fields_from_answer(self, answer, viz_id, widget_type=None):
        print(f"\n--- Extracting Fields for Viz_ID: {viz_id} ---")
        column_details = self._get_column_details_for_viz(viz_id)
        
        if not column_details:
            print(f"  WARNING: No column details found for viz {viz_id}. Using fallback.")
            return self._extract_fields_fallback(answer, viz_id, widget_type)
            
        fields = []
        answer_cols = answer.get('answer_columns', [])
        
        for col in answer_cols:
            original_col_name = col.get('name')
            if not original_col_name:
                continue
                
            if original_col_name in column_details:
                detail = column_details[original_col_name]
                
                # 1. Get Base Name & Expression
                field_name = detail['sanitized']
                
                # Use expression from table or default to agg(base)
                if detail['expression'] and str(detail['expression']).strip() != '':
                    field_expression = detail['expression']
                else:
                    agg = detail['aggregation']
                    base = detail['model_base_column']
                    field_expression = f"{agg}(`{base}`)" if agg else f"`{base}`"

                fields.append({
                    "name": field_name,           
                    "expression": field_expression 
                })
            else:
                # Fallback
                base_field = clean_field_name(original_col_name)
                sanitized_name = sanitize_alias_name(base_field)
                fields.append({
                    "name": sanitized_name, 
                    "expression": f"`{sanitized_name}`"
                })
                
        return fields
    
    def _extract_filters_for_query(self, viz_id):
        """
        Generates the 'filters' list for the Databricks query object 
        by looking up the viz_id in the loaded VIZ_FILTER_DATA table.
        """
        filters = []
        
        # Safety check: Ensure the table data was loaded
        if not hasattr(self, 'viz_filter_data') or self.viz_filter_data.empty:
            return []

        # 1. Filter the global table down to THIS specific viz_id
        current_viz_filters = self.viz_filter_data[
            self.viz_filter_data['viz_id'].astype(str) == str(viz_id)
        ]
        
        if current_viz_filters.empty:
            return []

        # 2. Iterate through valid filter rows for this viz
        for _, row in current_viz_filters.iterrows():
            filter_str = row.get('Filter_Details')
            col_name = row.get('Sanitized_Column')
            
            # Skip empty filters
            if not filter_str or pd.isna(filter_str) or str(filter_str).lower() in ['null', 'nan', 'none', '']:
                continue
                
            if not col_name:
                continue

            # 3. Generate SQL Condition
            # Use the sanitized column name from the table as the reference
            col_expr = f"`{col_name}`"
            sql_condition = self._get_sql_filter(col_expr, filter_str)
            
            if sql_condition:
                filters.append({
                    "expression": sql_condition
                })
                print(f"  ✓ Applied Filter to {col_name}: {sql_condition}")
        
        return filters

    def _extract_fields_fallback(self, answer, viz_id, widget_type=None):
        print(f"  INFO: Using fallback field extraction for viz {viz_id}")
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id)
        column_mapping = {}
        if mapping is not None:
            mapping_str_to_use = None
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() not in ['{}', '']:
                mapping_str_to_use = common_mapping_str
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                mapping_str_to_use = specific_mapping_str
            if mapping_str_to_use:
                try:
                    column_mapping = json.loads(mapping_str_to_use)
                except Exception as e:
                    print(f"  WARNING: Could not parse mapping: {e}")
        fields = []
        answer_cols = answer.get('answer_columns', [])
        for col in answer_cols:
            original_col_name = col.get('name')
            if not original_col_name:
                continue
            sql_alias = column_mapping.get(original_col_name)
            if not sql_alias:
                base_field = clean_field_name(original_col_name)
                sql_alias = column_mapping.get(base_field)
            if not sql_alias:
                base_field = clean_field_name(original_col_name)
                sql_alias = sanitize_alias_name(base_field)
            fields.append({"name": sql_alias, "expression": f"`{sql_alias}`"})
        return fields


    def _apply_expression_transformation(self, col_name, sql_alias, base_field, column_mapping):
        """
        Apply expression transformation using configuration from EXPRESSION_TRANSFORMATIONS.
        
        Args:
            col_name: Original column name from TML (e.g., "Day(Event Date)", "Total Revenue")
            sql_alias: Sanitized SQL alias (e.g., "Event_Date", "Revenue")
            base_field: Cleaned field name
            column_mapping: Dictionary of column mappings
            
        Returns:
            str: Transformed expression (e.g., "DATE_TRUNC('DAY', Event_Date)", "SUM(Revenue)")
        """
        import re
        
        # Try to match against configured expression patterns
        for transform in EXPRESSION_TRANSFORMATIONS:
            pattern = transform['pattern']
            target_expr = transform['target']
            
            # Handle date functions: Day(field), Month(field), etc.
            if pattern.startswith(('Day(', 'Week(', 'Month(', 'Year(')):
                func_match = re.match(r"(Day|Week|Month|Year)\((.*?)\)", col_name, re.IGNORECASE)
                if func_match:
                    func_name, field_name = func_match.groups()
                    db_field = column_mapping.get(field_name, field_name)
                    sanitized_field = sanitize_alias_name(base_field)
                    expression = target_expr.replace('field', sanitized_field)
                    expression = expression.replace('DAY', func_name.upper())
                    expression = expression.replace('WEEK', func_name.upper())
                    expression = expression.replace('MONTH', func_name.upper())
                    expression = expression.replace('YEAR', func_name.upper())
                    
                    return expression
            

            elif pattern.startswith(('sum(', 'count(', 'avg(', 'min(', 'max(')):
                continue
        
        return f'`{sql_alias}`'
        
    def _create_shared_dataset(self, viz, mapping):
        """
        Create a shared dataset using common SQL and column mapping from the mapping table.
        
        Args:
            viz: Visualization object (used for fallback display name)
            mapping: Mapping row with common_dataset_name, common_sql_query, common_column_mapping
            
        Returns:
            dict: Dataset object with name, displayName, and queryLines
        """
        dataset_name = mapping['common_dataset_name']
        
        # Get SQL query from mapping table
        if mapping.get('common_sql_query'):
            sql_query = mapping['common_sql_query']
            query_lines = sql_query.strip().split('\n')
        else:
            # Fallback: generate SQL from viz if common_sql_query is empty
            print(f"  WARNING: No common_sql_query found for shared dataset '{dataset_name}', generating from viz")
            query_lines = self._translate_search_query_to_sql(viz)
        

        display_name = mapping.get('common_dataset_name', viz.get('answer', {}).get('name', 'Shared Dataset'))
        
        print(f"  INFO: Shared dataset '{dataset_name}' created with {len(query_lines)} query lines")
        
        return {
            "name": dataset_name,
            "displayName": display_name,
            "queryLines": query_lines
        }
    def _get_base_frame(self, answer):
        return {
            "title": answer.get('name', 'Untitled'),
            "description": answer.get('description', ''),
            "showTitle": True,
            "showDescription": bool(answer.get('description'))
        }

    def _get_bar_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """Generates bar/column chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        colors = extract_colors_from_tml(chart) 
        mark = {}

        tml_type = chart.get('type', '')
        is_column = 'COLUMN' in tml_type.upper()
        is_stacked = 'STACKED' in tml_type.upper()

        print(f"\n=== {'COLUMN' if is_column else 'BAR'} CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}, Is Column: {is_column}, Is Stacked: {is_stacked}")

        # Extract Custom Axis Names ---
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")
        
        # --- Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id)
        column_mapping = {} # Holds the JSON mapping dict if found
        if mapping is not None:
            # Logic to load either common or specific mapping JSON
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: 
                    column_mapping = json.loads(mapping_str_to_use)
                    print(f"  Column mapping loaded: {column_mapping}")
                except (json.JSONDecodeError, TypeError) as e: 
                    print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: 
            print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        # --- Determine Axis Roles ---
        if is_column: category_axis_key, measure_axis_key = 'x', 'y'
        else: category_axis_key, measure_axis_key = 'y', 'x'

        tml_category_config = axis_config.get('x', [])
        tml_measure_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"Category Axis Key: {category_axis_key}, Measure Axis Key: {measure_axis_key}")
        print(f"TML Category Config: {tml_category_config}")
        print(f"TML Measure Config: {tml_measure_config}")
        print(f"TML Color Config: {tml_color_config}")

        # --- Process Category Axis (Dimension) ---
        if tml_category_config:
            original_field_cat = tml_category_config[0] # << CAPTURE ORIGINAL NAME FROM TML CONFIG
            matching_cat_col = next((col for col in answer_cols if col.get('name') == original_field_cat or clean_field_name(col.get('name')) == clean_field_name(original_field_cat)), None)

            if matching_cat_col:
                cat_col_name_from_answer = matching_cat_col.get('name')
                cat_base_field = clean_field_name(cat_col_name_from_answer)
                cat_sql_alias = column_mapping.get(original_field_cat, sanitize_alias_name(cat_base_field))
                print(f"Category axis ({category_axis_key}): TML={original_field_cat}, FoundInAnswer={cat_col_name_from_answer}, MappedSQLAlias={cat_sql_alias}")

                encodings[category_axis_key] = {
                    "fieldName": cat_sql_alias,                
                    "displayName": original_field_cat,         
                    "scale": {"type": infer_scale_type(cat_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_cat} 
                }
                available.append(f'{category_axis_key}-axis (category)')
            else:
                print(f"Category axis WARNING: No match in answer_cols for TML field '{original_field_cat}'")
                missing.append(f'{category_axis_key}-axis (category)')
        else:
             print(f"Category axis WARNING: No TML config found.")
             missing.append(f'{category_axis_key}-axis (category)')

        # --- Process Measure Axis ---
        measure_fields_processed = [] 
        original_measure_names = []  
        measure_axis_title = "Value"  

        if tml_measure_config:
            print(f"Processing {len(tml_measure_config)} measure field(s)...")
            for original_field_measure in tml_measure_config: 
                original_measure_names.append(original_field_measure)
                matching_measure_col = next((col for col in answer_cols if col.get('name') == original_field_measure or clean_field_name(col.get('name')) == clean_field_name(original_field_measure)), None)

                if matching_measure_col:
                    measure_col_name_from_answer = matching_measure_col.get('name')
                    measure_base_field = clean_field_name(measure_col_name_from_answer)
                    measure_sql_alias = column_mapping.get(original_field_measure, sanitize_alias_name(measure_base_field))
                    print(f"  - Measure field: TML={original_field_measure}, FoundInAnswer={measure_col_name_from_answer}, MappedSQLAlias={measure_sql_alias}")
                    measure_fields_processed.append({"fieldName": measure_sql_alias, "originalName": original_field_measure})
                else:
                    print(f"Measure axis WARNING: No match in answer_cols for TML field '{original_field_measure}'")
                    missing.append(f'{measure_axis_key}-axis measure: {original_field_measure}')

            # --- Construct Measure Axis Encoding ---
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                measure_axis_title = custom_y_axis_title or measure_info["originalName"] 
                encodings[measure_axis_key] = {
                    "fieldName": measure_info["fieldName"],    
                    "displayName": measure_info["originalName"], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": measure_axis_title}      
                }
                available.append(f'{measure_axis_key}-axis (single measure)')
                print(f"Measure axis ({measure_axis_key}): Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                measure_axis_title = custom_y_axis_title or ", ".join(original_measure_names[:3]) + ("..." if len(original_measure_names) > 3 else "")
                encodings[measure_axis_key] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": measure_axis_title} 
                }
                available.append(f'{measure_axis_key}-axis ({len(measure_fields_processed)} measures)')
                print(f"Measure axis ({measure_axis_key}): Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: 
                 if not any(m.startswith(f'{measure_axis_key}-axis measure:') for m in missing):
                    missing.append(f'{measure_axis_key}-axis (measure)')
        else:
             print(f"Measure axis WARNING: No TML config found.")
             missing.append(f'{measure_axis_key}-axis (measure)')

        # --- Process Color ---
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0]
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,        
                    "displayName": original_field_color, 
                    "scale": {"type": "categorical"}
                }
                
                color_dimension_present = True
                available.append('color')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             print("Color axis: No color dimension specified in TML.")

        # --- Determine Stacking/Grouping in `mark` ---
        if is_stacked:
            if color_dimension_present: 
                mark = {"stackType": "stacked"}
                available.append('stacking (by dimension)')
                print("Mark: Stacking by color dimension")
            elif len(measure_fields_processed) > 1: 
                mark = {"stackType": "stacked"}
                available.append('stacking (multiple measures)')
                print("Mark: Stacking by multiple measures")
            else: 
                print("Mark: Single measure, no color dimension - No stacking needed.")
        else: 
            if color_dimension_present: 
                mark = {"stackType": "grouped"}
                available.append('grouping (by dimension)')
                print("Mark: Grouping by color dimension")

        if not mark and not color_dimension_present and colors:
             print(f"Mark: Applying first TML color (fallback): {colors[0]}")
             mark = {"colors": colors[:1]}

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        return { "widgetType": "bar", "version": 3, "frame": self._get_base_frame(answer), "encodings": encodings, "mark": mark }

    def _get_line_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """Generates line chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        mark = {} 
        tml_type = chart.get('type', '')
        is_stacked_area = 'STACKED_AREA' in tml_type.upper()

        print(f"\n=== LINE/AREA CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}")

        # ---Extract Custom Axis Names ---
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")


        # --- Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id)
        column_mapping = {}
        if mapping is not None:
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: column_mapping = json.loads(mapping_str_to_use)
                except (json.JSONDecodeError, TypeError) as e: print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        # --- TML Config Extraction ---
        tml_x_config = axis_config.get('x', [])
        tml_y_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"TML X Config: {tml_x_config}")
        print(f"TML Y Config: {tml_y_config}")
        print(f"TML Color Config: {tml_color_config}")

        # --- Process X-axis (Dimension - usually temporal) ---
        if tml_x_config:
            original_field_x = tml_x_config[0] 
            matching_x_col = next((col for col in answer_cols if col.get('name') == original_field_x or clean_field_name(col.get('name')) == clean_field_name(original_field_x)), None)

            if matching_x_col:
                x_col_name_from_answer = matching_x_col.get('name')
                x_base_field = clean_field_name(x_col_name_from_answer)
                x_sql_alias = column_mapping.get(original_field_x, sanitize_alias_name(x_base_field))
                print(f"X-axis: TML={original_field_x}, FoundInAnswer={x_col_name_from_answer}, MappedSQLAlias={x_sql_alias}")

                encodings['x'] = {
                    "fieldName": x_sql_alias,
                    "displayName": original_field_x,
                    "scale": {"type": infer_scale_type(x_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_x}
                }
                available.append('x-axis')
            else:
                print(f"X-axis WARNING: No match in answer_cols for TML field '{original_field_x}'")
                missing.append('x-axis')
        else:
            print(f"X-axis WARNING: No TML config found.")
            missing.append('x-axis')

        # --- Process Y-axis (One or More Measures) ---
        measure_fields_processed = [] 
        original_y_names = []         
        y_axis_title = "Value"        

        if tml_y_config:
            print(f"Processing {len(tml_y_config)} Y-axis measure field(s)...")
            for original_field_y in tml_y_config: 
                original_y_names.append(original_field_y)
                matching_y_col = next((col for col in answer_cols if col.get('name') == original_field_y or clean_field_name(col.get('name')) == clean_field_name(original_field_y)), None)

                if matching_y_col:
                    y_col_name_from_answer = matching_y_col.get('name')
                    y_base_field = clean_field_name(y_col_name_from_answer)
                    y_sql_alias = column_mapping.get(original_field_y, sanitize_alias_name(y_base_field))
                    print(f"  - Y-axis measure: TML={original_field_y}, FoundInAnswer={y_col_name_from_answer}, MappedSQLAlias={y_sql_alias}")
                    measure_fields_processed.append({"fieldName": y_sql_alias, "originalName": original_field_y})
                else:
                    print(f"Y-axis WARNING: No match in answer_cols for TML field '{original_field_y}'")
                    missing.append(f'y-axis measure: {original_field_y}')

            # --- Construct Y-axis Encoding ---
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                y_axis_title = custom_y_axis_title or measure_info["originalName"] 
                encodings['y'] = {
                    "fieldName": measure_info["fieldName"],    
                    "displayName": measure_info["originalName"], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title}          
                }
                available.append('y-axis (single measure)')
                print(f"Y-axis: Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                y_axis_title = custom_y_axis_title or ", ".join(original_y_names[:3]) + ("..." if len(original_y_names) > 3 else "") 
                encodings['y'] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title} 
                }
                available.append(f'y-axis ({len(measure_fields_processed)} measures)')
                print(f"Y-axis: Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: 
                 if not any(m.startswith('y-axis measure:') for m in missing):
                    missing.append('y-axis (measure)')
        else:
             print(f"Y-axis WARNING: No TML config found.")
             missing.append('y-axis (measure)')

        # --- Process Color (Dimension for Multiple Lines) ---
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0] 
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,
                    "displayName": original_field_color, 
                    "scale": {"type": "categorical"}
                }
                color_dimension_present = True
                available.append('color (series)')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             if len(measure_fields_processed) > 1:
                 print("Color axis: No TML config, Databricks will use multiple Y-fields for color.")
                 available.append('color (implicit from measures)')
             else:
                 print("Color axis: No color dimension specified in TML.")
        
        # Handle Stacked Area
        if is_stacked_area:
             if color_dimension_present or len(measure_fields_processed) > 1:
                 mark = {"stackType": "stacked"}
                 available.append('stacking (area)')
                 print("Mark: Stacking enabled for area chart")

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        widget_type = "area" if "AREA" in tml_type.upper() else "line"

        return {
            "widgetType": widget_type,
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings,
            "mark": mark 
        }

    def _create_widget(self, viz, dataset_name):
        answer = viz.get('answer', {})
        chart = answer.get('chart', {})
        viz_id = viz.get('id')
        
        display_mode = answer.get('display_mode', '')
        tml_type = chart.get('type', 'TABLE_MODE' if display_mode == 'TABLE_MODE' else 'UNKNOWN')
        
        lvdash_type = TML_TO_LVDASH_MAPPING.get(tml_type)
        if lvdash_type is None:
            lvdash_type = 'table'
            status_note = f"Chart type '{tml_type}' has no direct mapping, using fallback"
        else:
            status_note = ""
        
        available = []
        missing = []
        unmapped_props = []
        
        # Get spec builder
        spec_builder = getattr(self, f'_get_{lvdash_type}_spec', self._get_table_spec)
        spec_result = spec_builder(answer, chart, available, missing, unmapped_props, viz.get('id'))
        
        # 1. Extract Fields
        fields = self._extract_fields_from_answer(answer, viz.get('id'))
        
        # 2. Extract Filters (NEW CALL)
        query_filters = self._extract_filters_for_query(viz.get('id'))
        
        # Check for Tuple (Counter + Trend)
        if isinstance(spec_result, tuple) and len(spec_result) == 2:
            counter_spec, line_spec = spec_result
            
            self.tracker.add_record(self.tml_filename, answer.get('name', 'Unnamed'), tml_type, 'counter', 'COMPLETE', available, missing, unmapped_props, status_note + " (with trend)")
            
            counter_widget = {
                "viz_id": viz_id,
                "viz_guid": viz.get('viz_guid'),
                "widget": {
                    "name": f"widget_{viz_id}_{generate_unique_id()}",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False,
                            "filters": query_filters # <--- INJECTED HERE
                        }
                    }],
                    "spec": counter_spec
                }
            }
            
            line_widget = {
                "viz_id": f"{viz_id}_trend",
                "viz_guid": viz.get('viz_guid'),
                "is_trend_chart": True,
                "parent_viz_id": viz_id,
                "widget": {
                    "name": f"widget_{viz_id}_trend_{generate_unique_id()}",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False,
                            "filters": query_filters # <--- INJECTED HERE
                        }
                    }],
                    "spec": line_spec
                }
            }
            return [counter_widget, line_widget]
        
        # Normal single widget handling
        self.tracker.add_record(self.tml_filename, answer.get('name', 'Unnamed'), tml_type, lvdash_type, 'COMPLETE', available, missing, unmapped_props, status_note)
        
        return {
            "viz_id": viz_id,
            "viz_guid": viz.get('viz_guid'),
            "widget": {
                "name": f"widget_{viz_id}_{generate_unique_id()}",
                "queries": [{
                    "name": "main_query",
                    "query": {
                        "datasetName": dataset_name,
                        "fields": fields,
                        "disaggregated": False,
                        "filters": query_filters # <--- INJECTED HERE
                    }
                }],
                "spec": spec_result  
            }
        }

    def _get_pie_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate pie chart specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Pie chart specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0] 
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass 
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process Y-axis (angle/value) 
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['angle'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('angle')
            else:
                missing.append('angle')
        else:
            missing.append('angle')
        
        # Process X-axis (color/category) 
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color')
            else:
                missing.append('color')
        else:
            missing.append('color')
        
        encodings['label'] = {"show": True}
        
        return {
            "widgetType": "pie", 
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_area_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None): 
        """Generates area/stacked area chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        mark = {} 
        tml_type = chart.get('type', '')
        is_stacked = 'STACKED' in tml_type.upper()

        print(f"\n=== AREA CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}, Is Stacked: {is_stacked}")

        #Extract Custom Axis Names 
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")

        # Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id)
        column_mapping = {}
        if mapping is not None:
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: column_mapping = json.loads(mapping_str_to_use)
                except (json.JSONDecodeError, TypeError) as e: print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        #TML Config Extraction
        tml_x_config = axis_config.get('x', [])
        tml_y_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"TML X Config: {tml_x_config}")
        print(f"TML Y Config: {tml_y_config}")
        print(f"TML Color Config: {tml_color_config}")

        #Process X-axis (Dimension)
        if tml_x_config:
            original_field_x = tml_x_config[0] 
            matching_x_col = next((col for col in answer_cols if col.get('name') == original_field_x or clean_field_name(col.get('name')) == clean_field_name(original_field_x)), None)

            if matching_x_col:
                x_col_name_from_answer = matching_x_col.get('name')
                x_base_field = clean_field_name(x_col_name_from_answer)
                x_sql_alias = column_mapping.get(original_field_x, sanitize_alias_name(x_base_field))
                print(f"X-axis: TML={original_field_x}, FoundInAnswer={x_col_name_from_answer}, MappedSQLAlias={x_sql_alias}")

                encodings['x'] = {
                    "fieldName": x_sql_alias,
                    "displayName": original_field_x, 
                    "scale": {"type": infer_scale_type(x_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_x} 
                }
                available.append('x-axis')
            else:
                print(f"X-axis WARNING: No match in answer_cols for TML field '{original_field_x}'")
                missing.append('x-axis')
        else:
            print(f"X-axis WARNING: No TML config found.")
            missing.append('x-axis')

        #Process Y-axis (One or More Measures)
        measure_fields_processed = [] 
        original_y_names = []         
        y_axis_title = "Value"        

        if tml_y_config:
            print(f"Processing {len(tml_y_config)} Y-axis measure field(s)...")
            for original_field_y in tml_y_config: 
                original_y_names.append(original_field_y)
                matching_y_col = next((col for col in answer_cols if col.get('name') == original_field_y or clean_field_name(col.get('name')) == clean_field_name(original_field_y)), None)

                if matching_y_col:
                    y_col_name_from_answer = matching_y_col.get('name')
                    y_base_field = clean_field_name(y_col_name_from_answer)
                    y_sql_alias = column_mapping.get(original_field_y, sanitize_alias_name(y_base_field))
                    print(f"  - Y-axis measure: TML={original_field_y}, FoundInAnswer={y_col_name_from_answer}, MappedSQLAlias={y_sql_alias}")
                    measure_fields_processed.append({"fieldName": y_sql_alias, "originalName": original_field_y})
                else:
                    print(f"Y-axis WARNING: No match in answer_cols for TML field '{original_field_y}'")
                    missing.append(f'y-axis measure: {original_field_y}')

            #Construct Y-axis Encoding
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                y_axis_title = custom_y_axis_title or measure_info["originalName"] 
                encodings['y'] = {
                    "fieldName": measure_info["fieldName"],    
                    "displayName": measure_info["originalName"], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title}          
                }
                available.append('y-axis (single measure)')
                print(f"Y-axis: Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                y_axis_title = custom_y_axis_title or ", ".join(original_y_names[:3]) + ("..." if len(original_y_names) > 3 else "") 
                encodings['y'] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title} 
                }
                available.append(f'y-axis ({len(measure_fields_processed)} measures)')
                print(f"Y-axis: Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: 
                 if not any(m.startswith('y-axis measure:') for m in missing):
                    missing.append('y-axis (measure)')
        else:
             print(f"Y-axis WARNING: No TML config found.")
             missing.append('y-axis (measure)')

        #Process Color (Dimension for Stacking)
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0] 
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,
                    "displayName": original_field_color, 
                    "scale": {"type": "categorical"}
                }
                color_dimension_present = True
                available.append('color (series)')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             if len(measure_fields_processed) > 1:
                 print("Color axis: No TML config, Databricks will use multiple Y-fields for color.")
                 available.append('color (implicit from measures)')
             else:
                 print("Color axis: No color dimension specified in TML.")
        
        # Handle Stacking
        if is_stacked:
             if color_dimension_present or len(measure_fields_processed) > 1:
                 mark = {"stackType": "stacked"}
                 available.append('stacking (area)')
                 print("Mark: Stacking enabled for area chart")

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        return {
            "widgetType": "area",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings,
            "mark": mark 
        }

    def _get_scatter_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate scatter plot specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Scatter plot specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass 
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)},
                    "axis": {"title": base_field}
                }
                available.append('x-axis')
            else:
                missing.append('x-axis')
        else:
            missing.append('x-axis')
        
        # Process Y-axis
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"},
                    "axis": {"title": base_field}
                }
                available.append('y-axis')
            else:
                missing.append('y-axis')
        else:
            missing.append('y-axis')
        
        # Process Color 
        if axis_config.get('color') and len(axis_config['color']) > 0:
            original_field = axis_config['color'][0]
            
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color')
        
        # Process Size 
        if axis_config.get('size') and len(axis_config['size']) > 0:
            original_field = axis_config['size'][0]
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['size'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('size')
        
        return {
            "widgetType": "scatter",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_table_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate table specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Table specification with widgetType, version, frame, and encodings
        """
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass 
        
        # Get ordered columns from table config or answer columns
        ordered_cols = answer.get('table', {}).get('ordered_column_ids', [])
        if not ordered_cols:
            ordered_cols = [c.get('name') for c in answer.get('answer_columns', []) if c.get('name')]
        
        answer_cols = answer.get('answer_columns', [])
        columns = []
        
        if ordered_cols:
            available.append('columns')
            for col_name in ordered_cols:
                if not col_name:
                    continue
                
                # Find matching answer column for proper type inference
                matching_col = None
                for col in answer_cols:
                    answer_col_name = col.get('name')
                    if answer_col_name == col_name or clean_field_name(answer_col_name) == clean_field_name(col_name):
                        matching_col = col
                        break
                
                if matching_col:
                    base_field = clean_field_name(matching_col.get('name'))
                    sql_alias = sanitize_alias_name(base_field)
                    
                    # Infer display type from column type
                    col_type = matching_col.get('type', 'string').lower()
                    display_as = "string"
                    if col_type in ['int', 'integer', 'long', 'bigint', 'float', 'double', 'decimal', 'numeric']:
                        display_as = "number"
                    elif col_type in ['date', 'timestamp', 'datetime']:
                        display_as = "datetime"
                    elif col_type in ['boolean', 'bool']:
                        display_as = "boolean"
                    
                    columns.append({
                        "fieldName": sql_alias,
                        "title": base_field,
                        "visible": True,
                        "alignContent": "left",
                        "allowHTML": False,
                        "displayAs": display_as,
                        "type": col_type if col_type else "string"
                    })
                else:
                    # Fallback if column not found in answer_columns
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    columns.append({
                        "fieldName": sql_alias,
                        "title": base_field,
                        "visible": True,
                        "alignContent": "left",
                        "allowHTML": False,
                        "displayAs": "string",
                        "type": "string"
                    })
        else:
            missing.append('columns')
        
        return {
            "widgetType": "table",
            "version": 1,
            "frame": self._get_base_frame(answer),
            "encodings": {"columns": columns},
            "allowHTMLByDefault": False,
            "itemsPerPage": 25
        }

    def _get_counter_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate counter/KPI specification with encodings matching SQL aliases.
        
        When temporal data is present, returns a TUPLE of (counter_spec, line_spec)
        to create both a KPI counter and a trend line chart below it.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict OR tuple: Counter spec, or (counter_spec, line_spec) if temporal data exists
        """
        widget_name = answer.get('name', 'Unknown')
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  
        
        # Get answer columns and axis config
        answer_cols = answer.get('answer_columns', [])
        axis_config = chart.get('axis_configs', [{}])[0] if chart.get('axis_configs') else {}
        
       
        has_temporal_field = False
        temporal_field_name = None
        if axis_config.get('x') and len(axis_config['x']) > 0:
            x_field = axis_config['x'][0]
            
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == x_field or clean_field_name(col_name) == clean_field_name(x_field):
                    if any(keyword in col_name for keyword in ['Month(', 'Day(', 'Week(', 'Year(', 'Date']):
                        has_temporal_field = True
                        temporal_field_name = col_name
                        print(f"  INFO: Detected temporal field '{temporal_field_name}' for KPI trend visualization")
                    break
        
        value_field_alias = None
        value_field_display = None
        
        if axis_config.get('y') and len(axis_config['y']) > 0:
            y_field = axis_config['y'][0]
            
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == y_field or clean_field_name(col_name) == clean_field_name(y_field):
                    base_field = clean_field_name(col_name)
                    value_field_alias = sanitize_alias_name(base_field)
                    value_field_display = base_field
                    break
        
        # If no value found from y-axis, find first non-temporal column
        if not value_field_alias and answer_cols:
            for col in answer_cols:
                col_name = col.get('name')
                if col_name:
                    if not any(keyword in col_name for keyword in ['Month(', 'Day(', 'Week(', 'Year(', 'Date']):
                        base_field = clean_field_name(col_name)
                        value_field_alias = sanitize_alias_name(base_field)
                        value_field_display = base_field
                        break
        
        if not value_field_alias:
            missing.append('value')
            return {
                "widgetType": "counter",
                "version": 2,
                "frame": self._get_base_frame(answer),
                "encodings": {}
            }
        
        # BUILD COUNTER SPEC 
        counter_encodings = {
            "value": {
                "fieldName": value_field_alias,
                "displayName": value_field_display or value_field_alias
            }
        }
        
        counter_spec = {
            "widgetType": "counter",
            "version": 2,
            "frame": self._get_base_frame(answer),
            "encodings": counter_encodings
        }
        
        available.append('value')
        
        # If we have temporal data, CREATE SEPARATE LINE CHART for trend
        if has_temporal_field and temporal_field_name:
            print(f"  INFO: Creating separate line chart for KPI trend using _get_line_spec")
            
            # Create separate lists for line chart tracking
            line_available = []
            line_missing = []
            line_unmapped = []
            
            # Call the existing line chart spec builder
            line_spec = self._get_line_spec(answer, chart, line_available, line_missing, line_unmapped, viz_id)
            
            # Customize the frame to hide title for trend chart
            line_spec['frame']['title'] = f"{answer.get('name', 'Trend')} - Trend"
            line_spec['frame']['showTitle'] = False  # Hide title for cleaner look
            
            available.extend(line_available)
            missing.extend(line_missing)
            unmapped_props.extend(line_unmapped)
            
            # Return TUPLE: (counter, line chart)
            return (counter_spec, line_spec)
        
        # No temporal data - return simple counter only
        return counter_spec

    def _get_heatmap_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate a normalized Databricks heatmap specification from a ThoughtSpot Answer.
        
        This version uses the RAW TML name for the user-visible legend title and
        axis display names, matching the TML exactly.
        
        Args:
            answer: Answer object containing chart data and columns.
            chart: Chart configuration with axis settings.
            available: List to track successfully mapped encodings.
            missing: List to track missing/failed encodings.
            unmapped_props: Unused parameter for consistency.
            viz_id: Optional visualization ID.
            
        Returns:
            dict: Databricks heatmap specification.
        """
        
        axis_config = chart.get('axis_configs', [{}])[0]
        answer_cols = answer.get('answer_columns', [])
        encodings = {}

        # --- Helper function for list-based axes (X and Y) ---
        def get_list_axis_details(tml_axis_key):
            """
            Finds column details for TML list-based axes (x, y).
            Returns (raw_display_name, field_name) or (None, None).
            """
            if column_mapping.get(tml_axis_key):
                field_name = column_mapping[tml_axis_key]
                matching_col = next(
                    (col for col in answer_cols if sanitize_alias_name(clean_field_name(col.get('name'))) == field_name), 
                    None
                )
                raw_display_name = matching_col.get('name') if matching_col else field_name
                return raw_display_name, field_name

            if axis_config.get(tml_axis_key) and len(axis_config[tml_axis_key]) > 0:
                original_field = axis_config[tml_axis_key][0]
                matching_col = next(
                    (col for col in answer_cols if 
                    col.get('name') == original_field or 
                    clean_field_name(col.get('name')) == clean_field_name(original_field)), 
                    None
                )
                if matching_col:
                    raw_display_name = matching_col.get('name')
                    field_name = sanitize_alias_name(clean_field_name(raw_display_name))
                    return raw_display_name, field_name
                    
            return None, None

        # --- Load Column Mapping ---
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass 

        # -- Process X-Axis ---
        x_display, x_field = get_list_axis_details("x")
        if x_field:
            encodings['x'] = {
                "fieldName": x_field,
                "displayName": x_display, 
                "scale": {
                    "type": infer_scale_type(x_display),
                    "sort": {"by": "natural-order"}
                }
            }
            available.append('x')
        else:
            missing.append('x')

        # -- Process Y-Axis ---
        y_display, y_field = get_list_axis_details("y")
        if y_field:
            encodings['y'] = {
                "fieldName": y_field,
                "displayName": y_display, 
                "scale": {
                    "type": infer_scale_type(y_display),
                    "sort": {"by": "natural-order-reversed"}
                }
            }
            available.append('y')
        else:
            missing.append('y')

        # --- Process Color/Measure 
        original_field = None
        measure_key = None
        color_field = None
        color_display = None
        # Check for mapping override first
        if column_mapping.get('color'):
            color_field = column_mapping['color']
            measure_key = 'color'
            matching_col = next((col for col in answer_cols if sanitize_alias_name(clean_field_name(col.get('name'))) == color_field), None)
            color_display = matching_col.get('name') if matching_col else color_field
        
        # If no override, parse TML
        if not color_field:
            if axis_config.get('color') and len(axis_config['color']) > 0:
                original_field = axis_config['color'][0]
                measure_key = 'color'
            elif axis_config.get('size') and isinstance(axis_config['size'], str):
                original_field = axis_config['size']
                measure_key = 'size'

            if original_field:
                matching_col = next(
                    (col for col in answer_cols if 
                    col.get('name') == original_field or 
                    clean_field_name(col.get('name')) == clean_field_name(original_field)), 
                    None
                )
                if matching_col:
                    color_display = matching_col.get('name')
                    color_field = sanitize_alias_name(clean_field_name(color_display))

        #-- Add to encodings if successful
        if color_field:
            encodings['color'] = {
                "fieldName": color_field, 
                "scale": {"type": "quantitative"},
                "legend": {
                    "hideTitle": False,
                    "title": color_display,
                    "position": "bottom",
                    "hide": False
                }
            }
            if measure_key:
                available.append(measure_key)
        else:
            missing.append('measure (color/size)')

        # -- Process Data Labels ---
        try:
            client_state_str = chart.get('client_state_v2', '{}')
            client_state = json.loads(client_state_str)
            chart_props = client_state.get('chartProperties', {})
            
            if chart_props.get('allLabels') is True:
                encodings['label'] = {"show": True}
                available.append('label')
                
        except (json.JSONDecodeError, TypeError):
            pass 

        # -- Return the DB Spec ---
        return {
            "widgetType": "heatmap",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_choropleth_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate choropleth (geo map) specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Choropleth specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis (geographic field)
        x_axis_config = axis_config.get('x')
        if x_axis_config and len(x_axis_config) > 0:
            original_field = x_axis_config[0]
            
            if original_field is None:
                missing.append('geographic field')
            else:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name is None:
                        continue
                    
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    sql_alias_lower = sql_alias.lower()
                    
                    if 'state' in sql_alias_lower or 'province' in sql_alias_lower:
                        if 'code' in sql_alias_lower or 'abbr' in sql_alias_lower or 'iso' in sql_alias_lower:
                            geo_role = "admin1-iso-a2"
                        else:
                            geo_role = "admin1-name"
                        
                        encodings['region'] = {
                            "regionType": "mapbox-v4-admin",
                            "admin1": {
                                "fieldName": sql_alias,
                                "type": "field",
                                "geographicRole": geo_role
                            }
                        }
                    else:
                        if 'iso2' in sql_alias_lower or sql_alias_lower.endswith('_2'):
                            geo_role = "admin0-iso-a2"
                        elif 'iso3' in sql_alias_lower or 'code' in sql_alias_lower or sql_alias_lower.endswith('_3'):
                            geo_role = "admin0-iso-3166-1-alpha-3"
                        else:
                            geo_role = "admin0-name"

                        encodings['region'] = {
                            "regionType": "mapbox-v4-admin",
                            "admin0": {
                                "fieldName": sql_alias,
                                "type": "field",
                                "geographicRole": geo_role
                            }
                        }
                    
                    available.append('geographic field')
                else:
                    missing.append('geographic field')
        else:
            missing.append('geographic field')
        
        # Process Y-axis (value field)
        y_axis_config = axis_config.get('y')
        if y_axis_config and len(y_axis_config) > 0:
            original_field = y_axis_config[0]
            
            if original_field is None:
                missing.append('value')
            else:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name is None:
                        continue
                    
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    encodings['color'] = {
                        "fieldName": sql_alias,
                        "scale": {
                            "type": "quantitative",
                            "colorRamp": {
                                "mode": "scheme",
                                "scheme": "blues"
                            }
                        }
                    }
                    available.append('value')
                else:
                    missing.append('value')
        else:
            missing.append('value')
        
        return {
            "widgetType": "choropleth-map",
            "version": 1,
            "frame": {
                "title": widget_name,
                "showTitle": True
            },
            "encodings": encodings
        }

    def _get_funnel_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate funnel specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Funnel specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis 
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('stage')
            else:
                missing.append('stage')
        else:
            missing.append('stage')
        
        # Process Y-axis
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('value')
            else:
                missing.append('value')
        else:
            missing.append('value')
        
        return {
            "widgetType": "funnel",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_sankey_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate Sankey diagram specification with encodings matching SQL aliases.
        
        Sankey charts require:
        - stages: Array of categorical fields representing the flow nodes
        - value: Quantitative field for flow thickness/weight
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Sankey specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        print(f"\n=== SANKEY CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
                print(f"Column mapping loaded: {column_mapping}")
            except (json.JSONDecodeError, TypeError) as e:
                print(f"Failed to parse column mapping: {e}")
        
        answer_cols = answer.get('answer_columns', [])
        
        # Collect stage fields (the flow nodes)
        stages = []
        
        # Process X-axis fields (typically source and intermediate nodes)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            for original_field in axis_config['x']:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    print(f"Stage field: original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                    
                    stages.append({
                        "fieldName": sql_alias
                    })
                else:
                    print(f"WARNING: Could not find matching column for stage field {original_field}")
        
        # Process Color axis fields 
        if axis_config.get('color') and len(axis_config['color']) > 0:
            for original_field in axis_config['color']:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    print(f"Stage field (from color): original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                    
                    # Avoid duplicates
                    if not any(s['fieldName'] == sql_alias for s in stages):
                        stages.append({
                            "fieldName": sql_alias
                        })
                else:
                    print(f"WARNING: Could not find matching column for stage field {original_field}")
        
        if stages:
            encodings['stages'] = stages
            available.append(f'stages ({len(stages)})')
            print(f"Added {len(stages)} stage fields")
        else:
            missing.append('stages')
            print("WARNING: No stage fields found for Sankey")
        
        # Process Y-axis
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                print(f"Value field: original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                
                encodings['value'] = {
                    "fieldName": sql_alias
                }
                available.append('value')
            else:
                print(f"WARNING: Could not find matching column for value field {original_field}")
                missing.append('value')
        else:
            missing.append('value')
        
        print(f"Final Sankey encodings: stages={[s['fieldName'] for s in encodings.get('stages', [])]}, value={encodings.get('value', {}).get('fieldName')}")
        print("="*50 + "\n")
        
        return {
            "widgetType": "sankey",
            "version": 1, 
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }


    def _get_pivot_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate pivot table specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: List to track optional unmapped properties
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Pivot table specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass 
        
        # Get ordered columns
        ordered_cols = answer.get('table', {}).get('ordered_column_ids', [])
        if not ordered_cols:
            ordered_cols = [c.get('name') for c in answer.get('answer_columns', []) if c.get('name')]
        
        answer_cols = answer.get('answer_columns', [])
        axis_config = chart.get('axis_configs', [{}])[0] if chart.get('axis_configs') else {}
        
        encodings = {}
        
        # Process Rows (x-axis)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            rows = []
            for original_field in axis_config['x']:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    rows.append({"fieldName": sql_alias, "displayName": base_field})
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    rows.append({"fieldName": sql_alias, "displayName": base_field})
            
            encodings['rows'] = rows
            available.append('rows')
        elif ordered_cols:
            original_field = ordered_cols[0]
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                encodings['rows'] = [{"fieldName": sql_alias, "displayName": base_field}]
            else:
                base_field = clean_field_name(original_field)
                sql_alias = sanitize_alias_name(base_field)
                encodings['rows'] = [{"fieldName": sql_alias, "displayName": base_field}]
            available.append('rows (inferred)')
        else:
            missing.append('rows')
        
        # Process Columns
        if axis_config.get('color') and len(axis_config['color']) > 0:
            columns = []
            for original_field in axis_config['color']:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    columns.append({"fieldName": sql_alias, "displayName": base_field})
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    columns.append({"fieldName": sql_alias, "displayName": base_field})
            
            encodings['columns'] = columns
            available.append('columns')
        else:
            unmapped_props.append('columns (pivot)')
        
        # Process Values (y-axis) 
        if axis_config.get('y') and len(axis_config['y']) > 0:
            fields = []
            for original_field in axis_config['y']:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    # Use the field name directly without wrapping in aggregation
                    fields.append({
                        "fieldName": sql_alias,  
                        "cellType": "text"
                    })
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias,  
                        "cellType": "text"
                    })
            
            encodings['cell'] = {
                "type": "multi-cell",
                "fields": fields
            }
            available.append('cell')
        elif len(ordered_cols) > 1:
            fields = []
            for original_field in ordered_cols[1:]:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias,  
                        "cellType": "text"
                    })
                else:
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias, 
                        "cellType": "text"
                    })
            
            encodings['cell'] = {
                "type": "multi-cell",
                "fields": fields
            }
            available.append('cell (inferred)')
        else:
            missing.append('cell')
        return {
        "widgetType": "pivot",
        "version": 3,
        "frame": self._get_base_frame(answer),
        "encodings": encodings
        }


    def _get_combo_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate combo chart specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: List to track optional unmapped properties
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Combo chart specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id)
        column_details = self._get_column_details_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]

            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)},
                    "axis": {"title": base_field}
                }
                available.append('x-axis')
            else:
                missing.append('x-axis')
        else:
            missing.append('x-axis')
        
        # Process Y-axis 
        if axis_config.get('y') and len(axis_config['y']) > 0:
            y_fields = axis_config['y']
            transformed_fields = []
            
            for original_field in y_fields:
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    transformed_fields.append(sql_alias)
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    transformed_fields.append(sql_alias)
            
            encodings['y'] = {
                "fieldName": transformed_fields,
                "displayName": "Value",
                "scale": {"type": "quantitative"},
                "axis": {"title": "Value"}
            }
            available.append('y-axis')
        else:
            missing.append('y-axis')
        
        # Process Color 
        if axis_config.get('color') and len(axis_config['color']) > 0:
            original_field = axis_config['color'][0]

            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color (series)')
        
        unmapped_props.append('series chart type mapping (combo charts partially supported)')
        
        return {
            "widgetType": "combo",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_sql_filter(self, col_expression, filter_str):
        """
        Converts TML shorthand into SQL conditions.
        Generic solution: Uses lower() for string comparisons to handle case mismatches.
        """
        if not filter_str or pd.isna(filter_str) or str(filter_str).lower() in ['nan', 'null', 'none']:
            return None
        s = str(filter_str).strip()

        # 1. Handle Null Checks
        if "!= '{null}'" in s: return f"{col_expression} IS NOT NULL"
        if "= '{null}'" in s: return f"{col_expression} IS NULL"

        # --- Helper to process individual values ---
        def process_val(val_chunk):
            val = val_chunk.strip()
            if val.startswith('.'): val = val[1:] # Remove leading dot
            
            # Remove existing quotes to check raw content
            clean_val = val.strip("'\"")
            
            # Check for Boolean/Numbers (Return raw value, IsString=False)
            if clean_val.lower() == 'true': return 'TRUE', False
            if clean_val.lower() == 'false': return 'FALSE', False
            if re.match(r'^\d+(\.\d+)?$', clean_val): return clean_val, False
            
            # It's a string: Lowercase it and quote it (Return value, IsString=True)
            return f"'{clean_val.lower()}'", True
        # -------------------------------------------

        # 2. Handle Lists (comma separated) -> IN clause
        if "," in s:
            parts = []
            has_strings = False
            # Split by comma
            raw_parts = s.split(',')
            
            for p in raw_parts:
                p_val, is_str = process_val(p)
                parts.append(p_val)
                if is_str: has_strings = True
            
            joined_values = ", ".join(parts)
            
            # Generic logic: if any value is a string, lower() the column
            if has_strings:
                return f"lower({col_expression}) IN ({joined_values})"
            else:
                return f"{col_expression} IN ({joined_values})"

        # 3. Handle Single Values (Dot notation or Operators)
        
        # Check standard operators first (>, <, =, !=)
        match_op = re.match(r'^(=|!=|<>|>|<|>=|<=)\s*(.*)', s)
        if match_op:
            op = match_op.group(1)
            raw_val = match_op.group(2)
            p_val, is_str = process_val(raw_val)
            
            if is_str:
                return f"lower({col_expression}) {op} {p_val}"
            else:
                return f"{col_expression} {op} {p_val}"
        
        # Default: Dot notation (Implicit IN/Equality)
        p_val, is_str = process_val(s)
        if is_str:
             return f"lower({col_expression}) IN ({p_val})"
        else:
             return f"{col_expression} IN ({p_val})"

        def clean_and_format_value(val_str):
            """Cleans value and applies mapping or title casing."""
            raw = val_str.strip()
            # Remove leading dot if present
            if raw.startswith('.'): raw = raw[1:]
            
            # Remove existing quotes for checking
            core_val = raw.strip("'\"")
            
            # Check for Boolean/Numbers first (return as-is, uppercase boolean)
            if core_val.lower() == 'true': return 'TRUE'
            if core_val.lower() == 'false': return 'FALSE'
            if re.match(r'^\d+(\.\d+)?$', core_val): return core_val
            
            # Handle Strings
            lower_val = core_val.lower()
            if lower_val in VALUE_MAPPING:
                return f"'{VALUE_MAPPING[lower_val]}'"
            
            # Fallback: Title Case (e.g. 'consumer' -> 'Consumer')
            # Note: This might capitalize 'in' -> 'In', which is why specific mapping above is preferred
            return f"'{core_val.title()}'"

        # 2. Handle Lists (comma separated) -> IN clause
        if "," in s:
            parts = [clean_and_format_value(p) for p in s.split(',')]
            return f"{col_expression} IN ({', '.join(parts)})"

        # 3. Handle Single Dot Values -> IN clause
        if s.startswith('.'):
            formatted_val = clean_and_format_value(s)
            return f"{col_expression} IN ({formatted_val})"

        # 4. Handle Standard Operators (>, =, !=)
        match = re.match(r'^(=|!=|<>|>|<|>=|<=)\s*(.*)', s)
        if match:
            op = match.group(1)
            raw_val = match.group(2)
            formatted_val = clean_and_format_value(raw_val)
            return f"{col_expression} {op} {formatted_val}"
            
        return None

    def _create_layout(self, layout_data, widgets):
        """
        Structured Hierarchy:
        1. Filters (Top - Height 1)
        2. KPI/Headline Cards (Middle - Height 2)
        3. Graphs/Charts (Bottom - Height 4)
        """
        layout = []
        GRID_WIDTH = 6
        
        # --- 1. CLASSIFICATION ---
        filters = [w for w in widgets if w.get('is_filter')]
        
        # Helper to identify Cards (KPIs) vs Graphs
        def is_card(widget_obj):
            if widget_obj.get('is_filter') or widget_obj.get('is_trend_chart'):
                return False
            spec = widget_obj.get('widget', {}).get('spec', {})
            w_type = str(spec.get('widgetType', '')).lower()
            return any(k in w_type for k in ['kpi', 'headline', 'counter'])
    
        cards = [w for w in widgets if is_card(w)]
        graphs = [w for w in widgets if not w.get('is_filter') and not is_card(w)]
    
        # --- 2. DIMENSIONS ---
        FILTER_W, FILTER_H = 2, 1  # 3 per row
        CARD_W, CARD_H     = 2, 2  # 3 per row
        GRAPH_W, GRAPH_H   = 3, 4  # 2 per row (Side-by-side)
    
        current_x = 0
        current_y = 0
    
        # --- 3. LAYER 1: FILTERS ---
        if filters:
            for f in filters:
                if current_x + FILTER_W > GRID_WIDTH:
                    current_x = 0
                    current_y += FILTER_H
                
                layout.append({
                    "widget": f['widget'],
                    "position": {"x": current_x, "y": current_y, "width": FILTER_W, "height": FILTER_H}
                })
                current_x += FILTER_W
            
            # Advance Y to next row and add a small spacer
            current_y += FILTER_H
            current_x = 0
    
        # --- 4. LAYER 2: KPI CARDS ---
        if cards:
            for card in cards:
                if current_x + CARD_W > GRID_WIDTH:
                    current_x = 0
                    current_y += CARD_H
                
                layout.append({
                    "widget": card['widget'],
                    "position": {"x": current_x, "y": current_y, "width": CARD_W, "height": CARD_H}
                })
                current_x += CARD_W
                
            # Advance Y to next row for graphs
            current_y += CARD_H
            current_x = 0
    
        # --- 5. LAYER 3: GRAPHS ---
        for graph in graphs:
            if current_x + GRAPH_W > GRID_WIDTH:
                current_x = 0
                current_y += GRAPH_H
            
            layout.append({
                "widget": graph['widget'],
                "position": {"x": current_x, "y": current_y, "width": GRAPH_W, "height": GRAPH_H}
            })
            current_x += GRAPH_W
    
        print(f"Layout Generated: {len(filters)} Filters, {len(cards)} Cards, {len(graphs)} Graphs.")
        return layout


    def _get_widget_size_from_db(self, size_category):
        """
        Fetch widget dimensions from pre-loaded WIDGET_SIZE_MAP.
        
        Args:
            size_category: Size category (e.g., 'SMALL', 'MEDIUM', 'LARGE')
            
        Returns:
            dict: Dictionary with 'width' and 'height' keys, or None if not found
        """
        if not size_category:
            print(f"WARNING: _get_widget_size_from_db called with no size category")
            return None
        
        # Use the pre-loaded WIDGET_SIZE_MAP from global scope
        size_config = WIDGET_SIZE_MAP.get(size_category)
        
        if size_config:
            print(f"INFO: Found size config for '{size_category}': width={size_config['width']}, height={size_config['height']}")
            return size_config
        
        print(f"WARNING: No size configuration found for category '{size_category}' in WIDGET_SIZE_MAP")
        print(f"Available size categories: {list(WIDGET_SIZE_MAP.keys())}")
        return None


    def _get_default_size_for_widget_type(self, widget_type):
        """
        Fetch default size category for a widget type from pre-loaded TML_TO_LVDASH_MAPPING.
        
        Args:
            widget_type: Widget type (e.g., 'bar', 'table', 'counter')
            
        Returns:
            str: Size category (e.g., 'MEDIUM'), or None if not found
        """
        normalized_widget_type = widget_type.replace(' ', '_')
        if not widget_type:
            print(f"WARNING: _get_default_size_for_widget_type called with no widget type")
            return None
        
        # Query the chart_type_mappings table for default_size
        try:
            from pyspark.sql import SparkSession
            spark = SparkSession.builder.getOrCreate()
            
            result = spark.sql(f"""
                SELECT default_size 
                FROM {CHART_TYPE_MAPPING_TABLE}
                WHERE widget_type = '{normalized_widget_type}'
                LIMIT 1
            """).collect()
            
            if result and len(result) > 0:
                default_size = result[0]['default_size']
                if default_size:
                    print(f"INFO: Found default size '{default_size}' for widget type '{normalized_widget_type}'")
                    return default_size
            
            print(f"WARNING: No default size found for widget type '{normalized_widget_type}'")
            print(f"Hint: Check if '{normalized_widget_type}' exists in {CHART_TYPE_MAPPING_TABLE}")
            return None
            
        except Exception as e:
            print(f"ERROR: Failed to fetch default size for widget type '{normalized_widget_type}': {e}")
            import traceback
            traceback.print_exc()
            return None

def convert_all_tml_files():
    setup_environment()
    
    # ===== Load liveboard configuration =====
    CONFIG_TABLE = f"{CATALOG}.{SCHEMA}.liveboard_migration_config"
    
    print("\n--- Loading Liveboard Configuration ---")
    try:
        config_df = spark.table(CONFIG_TABLE).toPandas()
        enabled_liveboards = config_df[config_df['process_flag'] == 'Y']
        
        if len(enabled_liveboards) == 0:
            print(f"WARNING: No liveboards enabled (process_flag='Y') in {CONFIG_TABLE}")
            print("Please enable at least one liveboard:")
            print(f"UPDATE {CONFIG_TABLE} SET process_flag = 'Y' WHERE name = 'Your Dashboard Name';")
            return
        
        enabled_names = set(enabled_liveboards['name'].tolist())
        enabled_guids = set(enabled_liveboards['guid'].tolist())
        
        print(f"Found {len(enabled_liveboards)} enabled liveboards:")
        for _, row in enabled_liveboards.iterrows():
            print(f"  ✓ {row['name']} ({row['guid']})")
        print("---\n")
        
    except Exception as e:
        print(f"ERROR: Could not load configuration table {CONFIG_TABLE}: {e}")
        print("Proceeding to convert ALL liveboards without filtering...")
        enabled_names = None
        enabled_guids = None

    
    if MAPPING_DATA is not None and len(MAPPING_DATA) > 0:
        total_mappings = len(MAPPING_DATA)
        mapped_count = len(MAPPING_DATA[
            (MAPPING_DATA['databricks_table_name_ToBeFilled'].notna()) & 
            (MAPPING_DATA['databricks_table_name_ToBeFilled'] != '')
        ])
        print(f"\n--- Mapping Status ---")
        print(f"Total visualizations: {total_mappings}")
        print(f"Mapped to Databricks tables: {mapped_count}")
        print(f"Unmapped (will use TML table names): {total_mappings - mapped_count}")
        print("---\n")
    
    try:
        tml_files = [f.path for f in dbutils.fs.ls(TML_INPUT_PATH) if f.path.endswith(('.tml', '.yaml', '.json'))]
    except Exception as e:
        print(f"ERROR: Cannot list files in '{TML_INPUT_PATH}'. Error: {e}")
        return

    if not tml_files:
        print(f"No TML files found in {TML_INPUT_PATH}")
        return
    
    # ===== Filter TML files based on enabled liveboards =====
    if enabled_names or enabled_guids:
        filtered_files = []
        skipped_files = []
        
        for file_path in tml_files:
            filename = Path(file_path).name
            
            try:
                tml_data = parse_tml_file(file_path)
                liveboard = tml_data.get('liveboard', {})
                lb_name = liveboard.get('name', '')
                lb_guid = tml_data.get('guid', '')
                
                if lb_name in enabled_names or lb_guid in enabled_guids:
                    filtered_files.append(file_path)
                    print(f"  ✓ Including: {filename} ({lb_name})")
                else:
                    skipped_files.append(filename)
                    
            except Exception as e:
                print(f"  WARNING: Could not parse {filename} for filtering: {e}")
                filtered_files.append(file_path)
        
        tml_files = filtered_files
        
        print(f"\n--- File Filtering Results ---")
        print(f"Total files found: {len(tml_files) + len(skipped_files)}")
        print(f"Files to process: {len(tml_files)}")
        print(f"Files skipped: {len(skipped_files)}")
        if skipped_files and len(skipped_files) <= 10:
            print(f"Skipped files: {', '.join(skipped_files)}")
        elif len(skipped_files) > 10:
            print(f"Skipped files: {', '.join(skipped_files[:10])}... and {len(skipped_files)-10} more")
        print("---\n")
    
    if not tml_files:
        print("No enabled TML files to process after filtering.")
        return
    
    print(f"Processing {len(tml_files)} enabled liveboard(s)...\n")

    
    tracker = ConversionTracker()
    summary_results = []
    failure_records = []
    
    for tml_file_path in tml_files:
        filename = Path(tml_file_path).name
        status = 'ERROR'
        lvdash_data = {}
        output_filename = 'N/A'
        
        try:
            print(f"\n--- Processing: {filename} ---")
            
            try:
                tml_data = parse_tml_file(tml_file_path)
            except Exception as parse_error:
                raise ValueError(f"Failed to parse TML file: {str(parse_error)}")
            
            converter = TMLToLVDASHConverter(
                tml_data, 
                filename, 
                tracker, 
                MAPPING_DATA, 
                COLUMN_DETAILS_DATA,
                VIZ_FILTER_DATA
            )
            lvdash_data = converter.convert()
            
            output_filename = filename.replace('.tml', '').replace('.yaml', '').replace('.json', '') + '.lvdash.json'
            output_path = os.path.join(LVDASH_OUTPUT_PATH, output_filename)
            dbutils.fs.put(output_path, json.dumps(lvdash_data, indent=2), overwrite=True)
            status = 'SUCCESS'
            print(f"  Saved to {output_filename}")

        except Exception as e:
            print(f"  FAILED: {e}")
            import traceback
            error_trace = traceback.format_exc()
            traceback.print_exc()
            
            failure_records.append({
                'tml_file': filename,
                'error_type': type(e).__name__,
                'error_message': str(e)[:1000],
                'stack_trace': error_trace[:2000],
                'failure_timestamp': datetime.now()
            })
        
        summary_results.append({
            'tml_file': filename, 
            'lvdash_file': output_filename, 
            'status': status,
            'num_datasets': len(lvdash_data.get('datasets', [])),
            'num_pages': len(lvdash_data.get('pages', [])),
            'num_widgets': sum(len(p.get('layout', [])) for p in lvdash_data.get('pages', [])),
            'conversion_timestamp': datetime.now()
        })

    print("\n--- Saving logs ---")
    if tracker.records:
        df = pd.DataFrame(tracker.records)
        df['conversion_timestamp'] = pd.to_datetime(df['conversion_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(TRACKER_TABLE)
        print(f"Saved {len(tracker.records)} widget records to {TRACKER_TABLE}")

    if summary_results:
        df = pd.DataFrame(summary_results)
        df['num_datasets'] = df['num_datasets'].astype('int32')
        df['num_pages'] = df['num_pages'].astype('int32')
        df['num_widgets'] = df['num_widgets'].astype('int32')
        df['conversion_timestamp'] = pd.to_datetime(df['conversion_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(SUMMARY_TABLE)
        print(f"Saved {len(summary_results)} file summaries to {SUMMARY_TABLE}")
    
    if failure_records:
        df = pd.DataFrame(failure_records)
        df['failure_timestamp'] = pd.to_datetime(df['failure_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(FAILURE_TABLE)
        print(f"Saved {len(failure_records)} failures to {FAILURE_TABLE}")
    
    print("\n--- Conversion complete! ---")

# --- EXECUTION ---
convert_all_tml_files()

print("--- Overall Conversion Summary ---")
try:
    display(spark.table(SUMMARY_TABLE).orderBy("conversion_timestamp", ascending=False))
except Exception as e:
    print(f"Could not display summary table. Error: {e}")

print("\n--- Detailed Widget Conversion Tracker ---")
try:
    display(spark.table(TRACKER_TABLE).orderBy("conversion_timestamp", ascending=False))
except Exception as e:
    print(f"Could not display tracker table. Error: {e}")

print("\n--- Conversion Failures ---")
try:
    fail_df = spark.table(FAILURE_TABLE)
    fail_count = fail_df.count()
    if fail_count > 0:
        print(f"Found {fail_count} failed conversions:")
        display(fail_df.orderBy("failure_timestamp", ascending=False))
    else:
        print("No failures!")
except Exception as e:
    print(f"Could not display failure table. Error: {e}")

print("\n--- Chart Type Mapping Statistics ---")
try:
    stats_query = f"""
    SELECT 
        tml_type,
        lvdash_type,
        status,
        COUNT(*) as count,
        COUNT(CASE WHEN unmapped_properties != '' THEN 1 END) as has_unmapped_props
    FROM {TRACKER_TABLE}
    GROUP BY tml_type, lvdash_type, status
    ORDER BY count DESC
    """
    display(spark.sql(stats_query))
except Exception as e:
    print(f"Could not display statistics. Error: {e}")

print("\n--- Unmapped Properties Report ---")
try:
    unmapped_query = f"""
    SELECT 
        tml_file,
        widget_name,
        tml_type,
        lvdash_type,
        unmapped_properties,
        notes
    FROM {TRACKER_TABLE}
    WHERE unmapped_properties != '' OR notes != ''
    ORDER BY tml_file, widget_name
    """
    display(spark.sql(unmapped_query))
except Exception as e:
    print(f"Could not display unmapped properties. Error: {e}")